# Exploratory Data Analysis   
__The following exploration is done to analyze which of the 2 cab companies, Yellow Company or Pink Company, is the better choice to invest in.__     

The following features are present in the dataset:    
1. txnID (Transaction ID): Unique id assigned to each of the rides that were done by either of the companies     
2. custID (Customer ID): Unique id assigned to each customer. Note that values in this field can be repeated as the same customer could have taken multiple rides with both the companies   
3. date (Date of Ride): The day of each month that the ride took place on   
4. month (Month of Ride): The month in which the ride took place   
5. year (Year of Ride): The year in which the ride took place    
6. day_name (Day Name): Name of the day on which the ride took place    
7. distance (Distance): The length/distance of each ride   
8. city (City): City in which the ride took place   
9. state (State): State of the city in which the ride took place    
10. cost (Cost): Cost incurred by each company for each ride   
11. price (Price): Price charged by each company for each ride  
12. profit (Profit): Profit earned by the company on each ride (Price-Cost)   
13. population (Population): Population of the city in which the ride took place   
14. user (User): Number of cab users that the city has    
15. pay_mode (Mode of Payment): The mode of payment that the customer used to pay for the ride   
16. gender (Gender): The gender of the customer    
17. age (Age): Age of the customer   
18. monthly_income (Monthly Income): Monthly income of the customer in US Dollars     19.
19. company (Company): Which company undertook the ride   
20. Holiday (Holiday): The name of the holiday on the date of the ride    

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [ ]:
df = pd.read_csv('Datasets/Master_Data.csv')
df.head()

### Hypotheses:   
1. Price has increased over the years   
2. Places with a higher population have higher prices and costs   
3. Profitability increases with larger duration    
4. Profit increases on Holidays    
5. People with higher income travel longer distances   
6. People in this age group travel more (22-40), due to absence of private modes of transport  

In [ ]:
df = df.set_index('txnID')
df.describe()

In [ ]:
df.info()

Here, we see that the Holiday feature has only 10906 not-null rows. While this might seem to be bad, it is useful information as it shows us that the ride took place on a day that wasn't a Federal US Holiday.

In [ ]:
corr = df.corr()
corr

In [ ]:
def get_grouped_df(df, indices, val, mean=False):
    if mean:
        grouped = pd.DataFrame(df.groupby(indices)[val].mean())
    else:
        grouped = pd.DataFrame(df.groupby(indices)[val].value_counts())
    df_list = []
    for index, row in grouped.iterrows():
        df_list.append([index[0], index[1], row[val]])
    group_df = pd.DataFrame(df_list)
    group_df.columns = indices+[val]
    return group_df

In [ ]:
group_df = get_grouped_df(df, ['company','year'], 'profit', mean=False)

In [ ]:
fig = plt.figure(figsize=(20,5))
sns.set_theme(style='whitegrid')
sns.countplot(y='year', hue='company', data=group_df).set_title('Number of Rides by each Company from 2016-2018')
plt.legend(title='')
plt.xlabel('Number of Rides')
plt.ylabel('')
plt.show()

In [ ]:
group_df = get_grouped_df(df, ['company','year'], 'profit', mean=True)
group_df.columns = ['Company','Year','Avg. Profit']
sns.set_theme(style='whitegrid')
g = sns.catplot(x='Year', y='Avg. Profit', hue='Company', data=group_df, kind='bar')
plt.title('Average Profit for Yellow and Pink Companies across 3 Years')
g.despine(left=True)
g.set_axis_labels('','Avg. Profit')
g.legend.set_title('')

__We can see that the average profits made by the Yellow company is almost double that of the Pink comapny in all 3 years. This, however, negates our first hypothesis (Price has increased over the years)__

In [ ]:
check = df.groupby(['company','month']).profit.count()
check

In [ ]:
months = ['January','February','March','April','May','June','July','August','September','October','November','December']
ylist = []
plist = []
for month in months:
    ylist.append(check.loc[('Yellow Cab', month)])
    plist.append(check.loc['Pink Cab', month])

In [ ]:
month_df = pd.DataFrame([months,plist,ylist])
month_df = month_df.T
month_df.columns = ['months','pink','yellow']
convert_dict = {'pink': int, 'yellow': int}
month_df = month_df.astype(convert_dict)
month_df.yellow.dtype
sns.lineplot(data=month_df, x='months', y='pink')
sns.lineplot(data=month_df, x='months', y='yellow')
plt.xlabel('Months')
plt.ylabel('Number of Rides')
plt.legend(['Pink Cab','Yellow Cab'])
plt.xticks(rotation=45)
plt.title('Number of Rides Each Month over 3 Years')
plt.show()

__We can see from the line plot that the sales of both the companies follow a similar pattern, with the yellow company having a much higher number.__

## Less in Jan coz of gyms and new years resolutions, etc.

__We will next analyze income and ride patterns seen in the data. For this analysis, it is assumed that that an income of greater than or equal to $8000/month is considered as high income.__

In [ ]:
df.head()

In [ ]:
df['income_level'] = df.monthly_income.apply(lambda x: 1 if x >= 12000 else 0)
df.income_level.value_counts()

In [ ]:
df.groupby('income_level').pay_mode.value_counts()

In [ ]:
table = pd.pivot_table(df, values=['cost','price','profit'], index=['company','city'], aggfunc=np.mean)

In [ ]:
# table = table.sort_values(table.index.name, ascending=False)
table

In [ ]:
def line_plot(data, x, y, hue):
    f, ax = plt.subplots(figsize=(10,5))
    g = sns.lineplot(data=data, x=x, y=y, hue=hue)
    y = y[0].upper() + y[1:]
    plt.title('Average {} of Each Ride in Different Cities'.format(y))
    plt.ylabel(y+' ($)')
    ax.legend().set_title('')
    plt.xticks(rotation=90)

In [ ]:
metrics = ['cost','price','profit']
sns.set_theme(style='white')
for metric in metrics:
    line_plot(table, 'city', metric, 'company')
    plt.xlabel('City')

In [ ]:
no_holiday_df = df[df.Holiday.isnull()]
holiday_df = df[~df.Holiday.isnull()]
hol_grp = holiday_df.groupby('company').profit.mean()
no_hol_grp = no_holiday_df.groupby('company').profit.mean()
hol_grp

In [ ]:
no_hol_grp

__ As we can observe from the above results, the average profit is higher on holidays than non-holidays for the Pink company and vice versa for the yellow company.__

In [ ]:
work_df = df[(df.age < 40) & (df.age > 22)]
no_work_df = df[(df.age >= 40) | (df.age <= 22)]
work_df.count()

In [ ]:
no_work_df.count()

In [ ]:
print(len(list(work_df.custID.unique())))
print(len(list(no_work_df.custID.unique())))